## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('chronic_kidney_disease_full.csv')

In [7]:
df.head(3)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.02,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.02,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
rbc      248 non-null object
pc       335 non-null object
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
sod      313 non-null float64
pot      312 non-null float64
hemo     348 non-null float64
pcv      329 non-null float64
wbcc     294 non-null float64
rbcc     269 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer: I might need to check if there is any collinearity among the predictores, such as pus cell and pus cell clumps

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [8]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [11]:
df.dropna().shape

(158, 25)

Answer: Only 158 rows remained in the data set if we dropped every row that contained at least one missing value.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer: There might be some measurement errors. For instance, nurses may read a wrong number when she/he is measuring the blood pressure. Or, the sphygmomanometer is not accurate.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: Null model. Simply tell every body that he/she has CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Sensitivity ( TP/(TP+FN) ) Since that everyone is told that he/she has CKD, so there is no False Negative, which makes recall = 1 ( TP/(TP+0) ). 
The false negatives are minimized to 0.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: Tell people who actually don't have CKD that he/she has CKD and without any scientific evidence support, which will cause them to be unnecessary panic.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: Another null model. Simply tell every one that she/he has CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Specificity( TN/(TN+FP) ). Minimized the false positves to 0.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: : Delayed or lack of supportive treatment if someone actually has CKD.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [16]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [18]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [ ]:
# Based on the article in pdf and information I got after doing some research. 
# I choose the following predictors: age,blood pressure, anemia,Pedal Edema,Sodium,Potassium,Red Blood Cell Count and White Blood Cell Count

In [28]:
df_new = df[['age','bp','ane','pe','sod','pot','wbcc','rbcc','class']]

In [29]:
df_new.head(3)

,age,bp,ane,pe,sod,pot,wbcc,rbcc,class
0,48.0,80.0,no,no,NaN,NaN,7800.0,5.2,ckd
1,7.0,50.0,no,no,NaN,NaN,6000.0,NaN,ckd
2,62.0,80.0,yes,no,NaN,NaN,7500.0,NaN,ckd


In [30]:
df_new = df_new.dropna()

In [54]:
X = df_new.drop('class',axis=1)
y = df_new['class']

sc = StandardScaler()

In [55]:
X = pd.get_dummies(X,drop_first=True)

X_sc = sc.fit_transform(X)

In [57]:
y = y.map(lambda x: 1 if x == 'ckd' else 0)

In [58]:
X_train,X_test,y_train,y_test = train_test_split(X_sc,y,random_state=42)

In [59]:
logreg = LogisticRegression()

logreg.fit(X_train,y_train)

LogisticRegression()

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [66]:
pd.DataFrame([X.columns,logreg.coef_.reshape(-1,1)]).T

,0,1
0,age,[0.5560033543146078]
1,bp,[1.0612321349254972]
2,sod,[-1.4522809966308665]
3,pot,[-0.07810009028653485]
4,wbcc,[0.9407273322035892]
5,rbcc,[-1.6991180667488934]
6,ane_yes,[0.7236993347142462]
7,pe_yes,[0.860600672449038]


In [ ]:
#### Answer:bp's coef is 1.061, which mean that for 1mm/Hg increase in blood pressure, the log(odds) of having ckd is crease 1.-61

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [ ]:
#### Answer: ane_yes:0.724 indicates that if some one has anemia then the log(odds) of having ckd increases by 0.724. 

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer:
* 1) Very easy to interpret
* 2) Very fast to calculate

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [67]:
### We can make two confusion matrix both on training and test to check if there is any problem such as overfitting.

In [69]:
predicted_train = logreg.predict(X_train)
predicted_test = logreg.predict(X_test)

train_conf = confusion_matrix(y_train,predicted_train)
test_conf = confusion_matrix(y_test,predicted_test)

In [70]:
train_conf

array([[91,  3],
       [ 9, 68]])

In [71]:
test_conf

array([[39,  1],
       [ 1, 17]])

### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: I think it's better to optimize sensitivity more than specificity. Because at least we won't delayed someone to get treatment if he/she really has CKD. 

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer: Firstly, the evaluation metric for unbalanced class data would not be accuracy. I think it is a problem in this CKD analysis.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: Suppose 99% of the observations in the dataset are having CKD, then the fitted model would predict almost all the unknown obervations to have CKD,

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [88]:
2/0.0000075

266666.6666666667

In [106]:
all_ckd = df_new[df_new['class'] == 'ckd']

In [107]:
all_ckd_sample = all_ckd.sample(266666,replace=True,random_state=42).reset_index(drop=True)

In [108]:
final_sample = pd.concat([all_ckd_sample,df_new[df_new['class'] != 'ckd'].sample(1,random_state=42)]).reset_index(drop=True)

In [109]:
final_sample['class'].value_counts()

ckd       266666
notckd         1
Name: class, dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [114]:
X_new = final_sample.drop('class',axis=1)
X_new = pd.get_dummies(X_new,drop_first=True)
X_new_sc = sc.fit_transform(X_new)

y_new = final_sample['class'].map(lambda x: 1 if x=='ckd' else 0)

logreg_1 = LogisticRegression()

In [118]:
logreg_1.fit(X_new_sc,y_new)

LogisticRegression()

In [120]:
new_pred = logreg_1.predict(X_test)

In [121]:
new_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [122]:
# every observations in test dataset have been predicted to be 1

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!